# Get managers for funds

In [ ]:
import tushare as ts
import pandas as pd
import time
from tqdm import tqdm
pro = ts.pro_api('5d2dd2c56bb822ac0e818aaa4b0b344f95c7d5e7b9c83c69e2ed90ff')

In [ ]:
fund_list = pd.read_csv("data/fund_list.csv", index_col=0)

For each funk, get it's manager. Only cconsider managers with 'begin_date' <= 2018. 

(There may be some logic mistakes in it.)

In [ ]:
def get_manager(fund_code, failed_times, success_times):
    for _ in range(3):
        try:
            df = pro.fund_manager(ts_code=fund_code)
            time.sleep(1)
        except:
            time.sleep(1)
        else:
            return df, failed_times
    failed_times += 1
    print(fund_code, "has failed. Total failed times is", failed_times, "success times is", success_times)
    return pd.DataFrame(columns = ['tmp']), failed_times

In [ ]:
success = 0
fail = 0
raw_manager_list = pd.DataFrame(columns = ['ts_code', 'name', 'birth_year', 'begin_date', 'end_date'])
for i in tqdm(fund_list['ts_code'], desc="get protfolio"):
    df, fail = get_manager(fund_code = i, failed_times = fail, success_times = success)
    if not df.empty:
        success += 1
        if df.iloc[-1]['begin_date'][:4] <= '2018':
            # Drop some useless columns.
            df = df.drop(columns=['gender', 'edu', 'nationality', 'ann_date', 'resume'])
            # Delete managers with 'end_date' < 20201231
            df = df.drop(df[df['end_date'] < '20201231'].index)
            # Add it to our manager list.
            raw_manager_list = pd.concat([raw_manager_list, df], ignore_index=True)

Store it in `manager_list_whole.csv`.

In [ ]:
raw_manager_list.to_csv("data/manager_list_whole.csv")